In [1]:
import qiskit
from qiskit import QuantumCircuit
import numpy as np
import copy
import time
from typing import Optional, List, Dict, Tuple, Union
from qiskit.result import Counts, Result
import pickle

from qiskit import pulse, IBMQ
from qiskit.result import marginal_counts


import logging
logger = logging.getLogger()

In [2]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
# Solvers
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver

# for HF calculation
from pyscf import scf
from pyscf import gto

from matplotlib import pyplot as plt

### Calculate eigen-energy given measurement in the Molecule pauli bases

In [3]:
# define relevant functions
def expval_with_variance(counts,
                         operator_coeff: int,
                         diagonal: Optional[np.ndarray] = None,
                        finite_sampling=True) -> Tuple[float, float]:
    r"""Compute the expectation value of a diagonal operator from counts.

    Args:
        counts: counts object.
        diagonal: Optional, values of the diagonal observable. If None the
                  observable is set to :math:`Z^\otimes n`.

    Returns:
        (float, float): the expectation value and variance.
    """
    if finite_sampling:
        # Get counts shots and probabilities
        probs = np.array(list(counts.values()))
        shots = probs.sum()
        probs = probs / shots
        
        # Get diagonal operator coefficients
        if diagonal is None:
            coeffs = np.array([(-1) ** (key.count('1') % 2)
                               for key in counts.keys()])
        else:
            keys = [int(key, 2) for key in counts.keys()]
            coeffs = np.asarray(diagonal[keys])
            
        # Compute expval
        expval = coeffs.dot(probs)
        
        # Compute variance
        if diagonal is None:
            # The square of the parity diagonal is the all 1 vector
            sq_expval = np.sum(probs)
        else:
            sq_expval = (coeffs ** 2).dot(probs)
        variance = (sq_expval - expval ** 2) / shots
    else:
        probs = counts

        # Get diagonal operator coefficients
        if diagonal is None:
            coeffs = np.array([(-1) ** (key.count('1') % 2)
                               for key in probs.keys()])
        else:
            keys = [int(key, 2) for key in probs.keys()]
            coeffs = np.asarray(diagonal[keys])

        # Compute expval
        expval = coeffs.dot(list(probs.values()))

        # Compute variance
        if diagonal is None:
            # The square of the parity diagonal is the all 1 vector
            sq_expval = np.sum(list(probs.values()))
        else:
            sq_expval = (coeffs ** 2).dot(list(probs.values()))
        variance = (sq_expval - expval ** 2)# / shots

    # Compute standard deviation
    if variance < 0:
        if not np.isclose(variance, 0):
            logger.warning(
                'Encountered a negative variance in expectation value calculation.'
                '(%f). Setting standard deviation of result to 0.', variance)
        variance = 0.0
    return expval * operator_coeff, variance * abs(operator_coeff) ** 2

def calculate_accumulate_exp_val(probs, coeffs, finite_sampling):
    combined_expval = 0.0
    combined_variance = 0.0
    if type(probs) != list:
        probs = [probs]
    for ind, result in enumerate(probs):
        exp_val, exp_var = expval_with_variance(result, coeffs[ind], finite_sampling=finite_sampling)
        # Accumulate
        combined_expval += exp_val
        combined_variance += exp_var
    combined_stddev = np.sqrt(combined_variance)
    return combined_expval, combined_stddev

### Measure the ansatz in all the given pauli bases

In [119]:
def create_circuits(paulis, ansatz, qubits):
    circuits = []
    for pauli in paulis:
        circuit = QuantumCircuit(2, 2)
        circuit.cx(0, 1)
        for i, val in enumerate(reversed(pauli)):
            if val == 'Y':
                circuit.sdg(i)
            if val in ['Y', 'X']:
                circuit.h(i)
            if val != 'I': # we calculate expectation value, so we do not need to measure the identity
                circuit.measure(i, i)
        circuit.add_calibration(
                        gate='cx',
                        qubits=qubits,
                        schedule=ansatz,
                    )
        circuits.append(circuit)
    return circuits

def paulis_expectation_values(counts, paulis, coeffs):
    for index, pauli in enumerate(paulis):
        marginal_qubits = None
        for i, val in enumerate(reversed(pauli)):
            if val == 'I': # we calculate expectation value, so we do not need to measure the identity
                marginal_qubits = 1 - i
        if marginal_qubits is not None:
            counts[index] = marginal_counts(counts[index], [marginal_qubits])
    expval, stddev = calculate_accumulate_exp_val(counts, coeffs, True)
    return expval, stddev
        

### the loss function - gets all the parameters and return the energy

In [64]:
def evaluate_energy_real_device(theta, paulis, coeffs, backend, 
                    dt=0.222, shots=10000, qubits=[5,6], nuclear_repulsion_energy=0):
    ansatz_sched = ansatz_gate_inspired_weighted_real_device(theta, dt, backend, qubits)
    
    circuits = create_circuits(paulis[3:], ansatz_sched, qubits)
    tran_circuits = qiskit.transpile(circuits, backend, initial_layout=qubits)
        
    print(theta)
    ansatz_job = backend.run(tran_circuits, shots=shots)
    # calculate expectation values
    results = ansatz_job.result()
    counts = results.get_counts()
    new_counts = [counts[0], counts[0], counts[0], counts[1]]
    expval, stddev = paulis_expectation_values(new_counts, paulis[1:], coeffs[1:])

    print("energy: " + str(expval.real + coeffs[0] + nuclear_repulsion_energy))
    # in case I will want to return also the varience and the counts
    # return results, expval, stddev
    
    # save in global variables the mid-optimization values
    thetas.append(copy.deepcopy(theta))
    values.append(expval)
    std_devs.append(stddev)
    energies.append(expval.real + coeffs[0] + nuclear_repulsion_energy)
    return expval.real

functions that already have the static parameters hard-coded for easier use

In [87]:
def calculate_energy_real_device(theta):
    paulis = all_paulis[-1]
    coeffs = all_coeffs[-1]
    nuclear_repulsion_energy = all_nuclear_repulsion_energies[-1]
    backend = backend_lagos
    dt = 0.222
    shots = 10000
    qubits = [5, 6]
    energy = evaluate_energy_real_device(theta, paulis, coeffs, backend, dt, shots, 
                                         qubits, nuclear_repulsion_energy)
    return energy

# Define the ansatz

In [122]:
def add_padding(waveform_samples, before=False):
    if len(waveform_samples) == 0:
        return waveform_samples, 0
    padding_to_add = 16 - (len(waveform_samples) % 16)
    if before:
        new_samples = [0.] * padding_to_add + waveform_samples
    else:
        new_samples = waveform_samples + [0.] * padding_to_add
    if len(new_samples) < 64:
        if before:
            new_samples = [0.] * (64 - len(new_samples)) + new_samples
        else:
            new_samples = new_samples + [0.] * (64 - len(new_samples))
    return new_samples, len(new_samples) - len(waveform_samples)

def amp_fun(pulse_duration, max_amp):
    return np.sign(pulse_duration) * max_amp #min(abs(pulse_duration) / 20, max_amp)

def ansatz_gate_inspired_weighted_real_device(theta, dt, backend, qubits=[5,6]):
    """
    theta - durations of drives in ns
    """
    control_weight = 2.5
    
    drive_zero = qubits[0]
    drive_one = qubits[1]
    backend_channels = backend.configuration().to_dict()['channels']
    for channel in backend_channels:
        if backend_channels[channel]['operates']['qubits'] == [qubits[0], qubits[1]]:
            control_zero = int(channel.split('u')[1])
    
    drive_0 = []
    drive_0_1 = []
    drive_1 = []
    drive_1_1 = []
    control_0 = []
    control_0_tag = []
    
    drag0_1_dur = 61 # found after doing some manual calibration on the device
    drag1_1_dur = 0

    control0_1_dur = 2 * int(control_weight * theta[0] / dt)

    drag0_2_dur = int(theta[1] / dt)
    drag1_2_dur = int(theta[2] / dt)
    # z rotations
    z0_1 = theta[3] * np.pi / 10 # 1/10 weight
    z1_1 = theta[4] * np.pi / 10 # 1/10 weight
    
    
    x_sigma = 64
    x_beta = -1.4
    x_amp = 0.2
    control_amp = 0.8
    control_width = abs(control0_1_dur/2)-14
    if control_width < 0:
        control_width = abs(control0_1_dur/2)
    
    qiskit.pulse.library.drag(16, 0.4, 64, -1.4).samples
    
    with pulse.build() as ansatz_sol_sched:
        flip_plus = list(qiskit.pulse.library.drag(abs(drag0_1_dur), 2*amp_fun(drag0_1_dur, x_amp),
                                                 x_sigma, x_beta).samples)
        flip_minus = list(qiskit.pulse.library.drag(abs(drag0_1_dur), -2*amp_fun(drag0_1_dur, x_amp),
                                                 x_sigma, x_beta).samples)
        
        if drag0_1_dur != 0:
            if abs(drag0_1_dur) > 32:
                drive_0 += list(qiskit.pulse.library.drag(abs(drag0_1_dur), amp_fun(drag0_1_dur, x_amp),
                                                 x_sigma, x_beta).samples)
            else:
                temp_width = abs(drag0_1_dur) - 14
                if temp_width < 0:
                    temp_width = abs(drag0_1_dur)
                drive_0 += list(qiskit.pulse.library.gaussian_square(duration=abs(drag0_1_dur), 
                                                                     amp=amp_fun(drag0_1_dur, x_amp),
                                                                     sigma=x_sigma, 
                                                                     width=temp_width).samples)
        if drag1_1_dur != 0:
            if abs(drag1_1_dur) > 32:
                drive_1 += list(qiskit.pulse.library.drag(abs(drag1_1_dur), amp_fun(drag1_1_dur, x_amp),
                                                 x_sigma, x_beta).samples)
            else:
                temp_width = abs(drag1_1_dur) - 14
                if temp_width < 0:
                    temp_width = abs(drag1_1_dur)
                drive_1 += list(qiskit.pulse.library.gaussian_square(duration=abs(drag1_1_dur), 
                                                                     amp=amp_fun(drag1_1_dur, x_amp),
                                                                     sigma=x_sigma, 
                                                                     width=temp_width).samples)
        if drag0_1_dur != 0 or drag1_1_dur != 0:
            delay_1 = abs(drag1_1_dur)
            if abs(drag0_1_dur) > abs(drag1_1_dur):
                delay_1 = abs(drag0_1_dur)
            else:
                delay_1 = abs(drag1_1_dur)
        else:
            delay_1 = 0
        padding_to_add_to_delay = 16 - (delay_1 % 16)
        delay_1 += padding_to_add_to_delay
        
        if control0_1_dur != 0:
            control_0 += list(qiskit.pulse.library.gaussian_square(duration=abs(int(control0_1_dur/2)), 
                                                          amp=amp_fun(control0_1_dur, control_amp), sigma=64,
                                                          width=control_width).samples)
            
            control_0_tag += list(qiskit.pulse.library.gaussian_square(duration=abs(int(control0_1_dur/2)), 
                                                          amp=-1*amp_fun(control0_1_dur, control_amp), sigma=64,
                                                          width=control_width).samples)
        if drag0_2_dur != 0:
            if abs(drag0_2_dur) > 32:
                drive_0_1 += list(qiskit.pulse.library.drag(abs(drag0_2_dur), amp_fun(drag0_2_dur, x_amp),
                                                   x_sigma, x_beta).samples)
            else:
                temp_width = abs(drag0_2_dur) - 14
                if temp_width < 0:
                    temp_width = abs(drag0_2_dur)
                drive_0_1 += list(qiskit.pulse.library.gaussian_square(duration=abs(drag0_2_dur), 
                                                                       amp=amp_fun(drag0_2_dur, x_amp),
                                                                       sigma= x_sigma, 
                                                                       width=temp_width).samples)
        if drag1_2_dur != 0:
            if abs(drag1_2_dur) > 32:
                drive_1_1 += list(qiskit.pulse.library.drag(abs(drag1_2_dur), amp_fun(drag1_2_dur, x_amp),
                                                   x_sigma, x_beta).samples)
            else:
                temp_width = abs(drag1_2_dur) - 14
                if temp_width < 0:
                    temp_width = abs(drag1_2_dur)
                drive_1_1 += list(qiskit.pulse.library.gaussian_square(duration=abs(drag1_2_dur), 
                                                                       amp=amp_fun(drag1_2_dur, x_amp),
                                                                       sigma=x_sigma, 
                                                                       width=temp_width).samples)
        
        # add padding to all pulses
        drive_0, _ = add_padding(drive_0)
        drive_0_1, drive_0_back_added = add_padding(drive_0_1)
        drive_1, _ = add_padding(drive_1)
        drive_1_1, drive_1_back_added = add_padding(drive_1_1)
        control_0, _ = add_padding(control_0)
        control_0_tag, _ = add_padding(control_0_tag)
        flip_plus, _ = add_padding(flip_plus)
        flip_minus, _ = add_padding(flip_minus)
        # play all the pulses
        if len(drive_0) != 0:
            pulse.play(pulse.Waveform(drive_0), pulse.DriveChannel(drive_zero))
        if len(drive_1) != 0:
            pulse.play(pulse.Waveform(drive_1), pulse.DriveChannel(drive_one))
        pulse.delay(delay_1, pulse.ControlChannel(control_zero))
        if len(control_0) != 0:
            pulse.play(pulse.Waveform(control_0), pulse.ControlChannel(control_zero))
            pulse.delay(delay_1 + len(control_0) - len(drive_0), pulse.DriveChannel(drive_zero))
            pulse.play(pulse.Waveform(flip_plus), pulse.DriveChannel(drive_zero))
            pulse.delay(len(flip_plus), pulse.ControlChannel(control_zero))
            pulse.play(pulse.Waveform(control_0_tag), pulse.ControlChannel(control_zero))
            pulse.delay(len(control_0_tag), pulse.DriveChannel(drive_zero))
            pulse.play(pulse.Waveform(flip_minus), pulse.DriveChannel(drive_zero))

        pulse.shift_phase(z0_1, pulse.DriveChannel(drive_zero))
        if len(drive_0_1) != 0:
            pulse.play(pulse.Waveform(drive_0_1), pulse.DriveChannel(drive_zero))
        delay_drive_1 = delay_1 + len(control_0) + len(control_0_tag) - len(drive_1) \
            + len(flip_plus) + len(flip_minus) #- drive_1_back_added
        if delay_drive_1 > 0:
            pulse.delay(delay_drive_1, pulse.DriveChannel(drive_one))
        pulse.shift_phase(z1_1, pulse.DriveChannel(drive_one))
        if len(drive_1_1) != 0:
            pulse.play(pulse.Waveform(drive_1_1), pulse.DriveChannel(drive_one))

    return ansatz_sol_sched

# Functions to build the problem parameters

In [11]:
def get_qubit_op(dist, mapper="parity"):
    molecule = Molecule(geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, dist]]], charge=0, multiplicity=1)
    driver = ElectronicStructureMoleculeDriver(
        molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF)
    es_problem = ElectronicStructureProblem(driver)
    second_q_op = es_problem.second_q_ops()
    if mapper == "parity":
        qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
    elif mapper == "jordan":
        qubit_converter = QubitConverter(mapper=JordanWignerMapper(), two_qubit_reduction=True)
    elif mapper == "bravi":
        qubit_converter = QubitConverter(mapper=BravyiKitaevMapper(), two_qubit_reduction=True)
    else:
        print("mapper should be one of parity, jordan or bravi")
        return None
    qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
    return qubit_op, es_problem

In [12]:
def get_hf_energy(dist):
    temp_mol = gto.M(atom = 'H 0 0 0; H 0 0 ' + str(dist), basis = 'sto3g')
    temp_hf = scf.HF(temp_mol)
    return temp_hf.kernel()

In [13]:
def get_accurate_state(dist):
    temp_qubit_op = get_qubit_op(dist)[0]
    eigen_values, eigen_vectors = np.linalg.eigh(temp_qubit_op.to_matrix())
    min_eigen_value = eigen_values[0]
    min_eigen_vector = eigen_vectors.transpose()[0]
    return min_eigen_value, min_eigen_vector

In [14]:
def get_exact_energies(dist):
    numpy_solver = NumPyMinimumEigensolver()
    qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
    calc = GroundStateEigensolver(qubit_converter, numpy_solver)
    _, es_prob = get_qubit_op(dist)
    temp_res = calc.solve(es_prob)
    return temp_res, temp_res.total_energies[0], temp_res.nuclear_repulsion_energy

In [15]:
def get_paulis_and_coeffs(dist):
    qubit_op = get_qubit_op(dist)[0]
    paulis = []
    for op in qubit_op:
        paulis.append(str(op)[-op.num_qubits:])
    return paulis, qubit_op.coeffs

# Visualization functions

In [16]:
def plot_energies(energies, exp_energies, distances):
    fig = plt.figure(figsize=(8, 5))
    plt.plot(distances, energies, lw=3, label="Exact Energy")
    plt.plot(distances, exp_energies, lw=3, label="Exp Energy")
    plt.fill_between(distances, np.array(energies)-0.0016, np.array(energies)+0.0016, 
                     alpha=0.2, facecolor='#089FFF', label="Chemical accuracy")
    plt.xlabel('Atomic distance (Angstrom)')
    plt.ylabel('Energy')
    plt.legend()
    plt.show()

In [17]:
def plot_energies_error(energies, exp_energies, distances):
    fig = plt.figure(figsize=(8, 5))
    plt.plot(distances, np.abs(np.array(energies) - np.array(exp_energies)), lw=3, label="Exp Error")
    plt.axhline(0.0016, linestyle='--', label="Chemical accuracy")
    plt.xlabel('Atomic distance (Angstrom)')
    plt.ylabel('Energy Error')
    plt.legend()
    plt.show()

In [54]:
def plot_convergence(values_hist_arr, dist):
    values_hist_arr_real = np.array(values_hist_arr[dist])
    paulis, coeffs = get_paulis_and_coeffs(dist)
    values_hist_arr_real = values_hist_arr_real + coeffs[0]
    values_hist_arr_real = np.array([spsa_value.real for spsa_value in values_hist_arr_real])
    
    temp_res, min_energy, nuclear_repulsion_energy = get_exact_energies(dist)
    min_eigenvalue = min_energy - nuclear_repulsion_energy

    fig = plt.figure(figsize=(8, 5))
    plt.plot(list(range(len(values_hist_arr[dist]))), values_hist_arr_real , lw=3, label="energy")
    plt.plot(list(range(len(values_hist_arr[dist]))), [min_eigenvalue] * len(values_hist_arr_real) , 'k', lw=3, 
             label="optimal energy", linestyle="dashed")
    plt.xlabel("iteration")
    plt.ylabel("Energy")
    plt.legend(frameon=False)
    plt.title("H_2 convergence")
    plt.show()

# Run the optimization

some experiments ran with the code.

The final results which were presented in the paper were ran on dedicated server. after the tests here, a single run  (with multiple iterations) was done to collect the final data.

In [85]:
token = "" # fill your user token here
provider = IBMQ.save_account(token, overwrite=True)

In [120]:
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research-2', group='technion-4', project='main') # use your provider here
backend_lagos = provider.get_backend('ibm_lagos')

ibmqfactory.load_account:WARNING:2022-09-04 00:58:16,150: Credentials are already in use. The existing account in the session will be replaced.


In [123]:
dists = [0.3, 0.5, 0.6]

thetas_hist1 = {}
values_hist1 = {}
std_devs_hist1 = {}
energies_hist1 = {}
hill_step_run_results1 = {}
    
all_paulis = []
all_coeffs = []
all_nuclear_repulsion_energies = []

backend = backend_lagos

for dist in dists:
    print("#########################################")
    print("Solving for distance " + str(dist) + ":")
    print("#########################################")
    paulis, coeffs = get_paulis_and_coeffs(dist)
    all_paulis.append(paulis)
    all_coeffs.append(coeffs)
    temp_res, min_energy, nuclear_repulsion_energy = get_exact_energies(dist)
    all_nuclear_repulsion_energies.append(nuclear_repulsion_energy)
    target_energy = min_energy

    initial_theta = [10 * dt, 62 * dt, 0, 0., 0.]

    
    parameter_search_space = {
        "0": np.linspace(20, -20, 21),
        "1": np.linspace(30, -30, 21),
        "2": np.linspace(30, -30, 21),
        "3": np.linspace(4, -4, 17),
        "4": np.linspace(4, -4, 17)
    }    
    
    parameters_order = [0, 1, 4, 3, 2]

    thetas = []
    values = []
    std_devs = []
    energies = []

    tic = time.perf_counter()
    
    
    cur_hill_step_run_results = []   

    cur_theta = copy.deepcopy(initial_theta)
    cur_energy = 10
    
    for i in range(4):
        if i >= 2:
            parameter_search_space = {
                "0": np.linspace(5, -5, 11),
                "1": np.linspace(5, -5, 11),
                "2": np.linspace(5, -5, 11),
                "3": np.linspace(0.5, -0.5, 11),
                "4": np.linspace(0.5, -0.5, 11)
            }
        for param_index in parameters_order:
            cur_hill_step_results = {}
            cur_energies = []
            cur_std_devs = []
            test_circs = []
            print("theta: " + str(cur_theta))
            print("working on theta[" + str(param_index) + "]")
            for param in parameter_search_space[str(param_index)]:
                test_theta = copy.deepcopy(cur_theta)
                if param_index in [0, 1, 2]:
                    param = param * dt
                test_theta[param_index] += param
                test_ansatz = ansatz_gate_inspired_weighted_real_device(test_theta, 0.222, backend, [5,6])
                circuits = create_circuits(paulis[3:], test_ansatz, [5,6])
                test_circs += circuits
            tran_circ_lagos = qiskit.transpile(test_circs, backend, initial_layout=[5,6])
            test_job_lagos = backend.run(tran_circ_lagos, shots=10000)
            cur_job_id = test_job_lagos.job_id()
            print("job id: " + str(cur_job_id))
            cur_hill_step_results["job_id"] = cur_job_id
            cur_results = test_job_lagos.result()
            cur_counts = cur_results.get_counts()
            cur_hill_step_results["counts"] = cur_counts
            for i in range(0, len(cur_counts), 2):
                temp_counts = [cur_counts[i], cur_counts[i], cur_counts[i], cur_counts[i+1]]
                expval, stddev = paulis_expectation_values(temp_counts, paulis[1:], coeffs[1:])
                print("energy: " + str(expval.real + coeffs[0] + nuclear_repulsion_energy))
                cur_energies.append(expval.real + coeffs[0] + nuclear_repulsion_energy)
                cur_std_devs.append(stddev)
            
            cur_hill_step_results["energies"] = cur_energies
            temp_min_energy = np.min(cur_energies)
            print("min energy found: " + str(temp_min_energy))
            cur_hill_step_results["min_energy"] = temp_min_energy
            opt_energy_index = cur_energies.index(temp_min_energy)
            cur_hill_step_results["std"] = cur_std_devs[opt_energy_index]
            if temp_min_energy < cur_energy:
                cur_energy = temp_min_energy
                if param_index in [0, 1, 2]:
                    multiplier = dt
                else:
                    multiplier = 1
                cur_theta[param_index] = cur_theta[param_index] + \
                    parameter_search_space[str(param_index)][opt_energy_index] * multiplier
            cur_hill_step_results["theta"] = copy.deepcopy(cur_theta)
            print("updated theta: " + str(cur_theta))
            # save results
            values.append(temp_min_energy - coeffs[0] - nuclear_repulsion_energy)
            energies.append(temp_min_energy)
            std_devs.append(cur_std_devs[opt_energy_index])
            thetas.append(copy.deepcopy(cur_theta))
            cur_hill_step_run_results.append(cur_hill_step_results)
    
    toc = time.perf_counter()

    thetas_hist1[dist] = copy.deepcopy(thetas)
    values_hist1[dist] = copy.deepcopy(values)
    std_devs_hist1[dist] = copy.deepcopy(std_devs)
    energies_hist1[dist] = copy.deepcopy(energies)
    hill_step_run_results1[dist] = copy.deepcopy(cur_hill_step_run_results)

    opt_theta_index = values.index(np.min(values))

    print(f"Finished the VQE in {toc - tic:0.4f} seconds")
    print("final theta: " + str(thetas[opt_theta_index]))
    res = min(energies)
    print("min eigen_value found: " + str(res))
    print("absolute error: " + str(np.abs(res - target_energy)))
    print("std: " + str(std_devs[opt_theta_index]))
    print("number of iterations: " + str(i))
    opt_sched = ansatz_gate_inspired_weighted_real_device(thetas[opt_theta_index], dt,
                                                      backend, qubits)
    print("solution schedule duration: " + str(opt_sched.duration * dt) + " [ns]")
    
all_results = {"thetas": thetas_hist1,
              "values": values_hist1,
              "std_devs": std_devs_hist1,
              "energies": energies_hist1,
              "run_results": hill_step_run_results1}

with open("./h2_results_hill_lagos1.pkl", 'ab') as pickle_results_file:
    pickle.dump(all_results, pickle_results_file)

#########################################
Solving for distance 0.3:
#########################################
theta: [2.22, 13.764, 0, 0.0, 0.0]
working on theta[0]
job id: 63147a346acc638139372751
energy: (-0.42742023117949346+0j)
energy: (-0.47091842529847616+0j)
energy: (-0.4866188639021187+0j)
energy: (-0.5307138315511519+0j)
energy: (-0.11595080769225219+0j)
energy: (0.3503339009409312+0j)
energy: (0.0717401410003311+0j)
energy: (-0.3450148698977573+0j)
energy: (-0.5323435996349093+0j)
energy: (-0.5454342070941087+0j)
energy: (-0.4883989418404453+0j)
energy: (-0.41204748018343773+0j)
energy: (-0.4561116949482611+0j)
energy: (-0.5176548190703114+0j)
energy: (-0.4402083059416606+0j)
energy: (-0.5548479351112028+0j)
energy: (-0.46787625389374954+0j)
energy: (-0.3135334506085887+0j)
energy: (-0.3545382659870975+0j)
energy: (-0.4732853617603223+0j)
energy: (-0.5329519653721784+0j)
min energy found: (-0.5548479351112028+0j)
updated theta: [0.0, 13.764, 0, 0.0, 0.0]
theta: [0.0, 13.764, 

job id: 63147f5530de607adf3c2117
energy: (-0.41616857222698167+0j)
energy: (-0.5071973607276195+0j)
energy: (-0.3648903788081346+0j)
energy: (-0.4341085122254049+0j)
energy: (-0.5574702191628735+0j)
energy: (-0.5595663274891916+0j)
energy: (-0.5487020392428228+0j)
energy: (-0.3310723483493856+0j)
energy: (-0.2450926374672484+0j)
energy: (-0.34469229898195897+0j)
energy: (-0.396059541024274+0j)
min energy found: (-0.5595663274891916+0j)
updated theta: [0.0, 13.764, -0.666, 0.0, 1.5]
theta: [0.0, 13.764, -0.666, 0.0, 1.5]
working on theta[1]
job id: 63147fa4c7ab17dd3e2b4c66
energy: (-0.5540701136977009+0j)
energy: (-0.5485961629446967+0j)
energy: (-0.5501207348770976+0j)
energy: (-0.5573783897187838+0j)
energy: (-0.5577766546862701+0j)
energy: (-0.5584309078964984+0j)
energy: (-0.5530092126085262+0j)
energy: (-0.5575322991003429+0j)
energy: (-0.5502010078572872+0j)
energy: (-0.5494805285209057+0j)
energy: (-0.5508007581322718+0j)
min energy found: (-0.5584309078964984+0j)
updated theta: 

job id: 631530eae85847f218772b1a
energy: (-0.681369461726719+0j)
energy: (-0.7510571436118283+0j)
energy: (-0.8280958680585395+0j)
energy: (-0.8898320358050194+0j)
energy: (-0.9398227919672852+0j)
energy: (-0.9793225601795676+0j)
energy: (-1.0051010461459564+0j)
energy: (-1.016598840728032+0j)
energy: (-1.0152268953631436+0j)
energy: (-0.9960659851932132+0j)
energy: (-0.9681467155704977+0j)
energy: (-0.9367113914285012+0j)
energy: (-0.8844670109340254+0j)
energy: (-0.818670312822954+0j)
energy: (-0.7487707499457845+0j)
energy: (-0.6579614849423865+0j)
energy: (-0.5798247839017316+0j)
min energy found: (-1.016598840728032+0j)
updated theta: [0.0, 13.764, 0, 0.0, -1.0]
theta: [0.0, 13.764, 0, 0.0, -1.0]
working on theta[2]
job id: 631534155a9b0e57e4a6d401
energy: (-0.7842550990357526+0j)
energy: (-0.8430457279979884+0j)
energy: (-0.8736375140790427+0j)
energy: (-0.9121440425878347+0j)
energy: (-0.942175100108704+0j)
energy: (-0.9781888532375083+0j)
energy: (-0.9711588423620003+0j)
energy

job id: 63161ea276e4d119f70b40b9
energy: (-0.915204325745838+0j)
energy: (-0.9733639006999126+0j)
energy: (-0.8538590642248518+0j)
energy: (-0.9071300939845728+0j)
energy: (-1.0169319390945615+0j)
energy: (-1.018267970453952+0j)
energy: (-1.0167747212898013+0j)
energy: (-0.8141043098545302+0j)
energy: (-0.7963902233305407+0j)
energy: (-0.8746525555492453+0j)
energy: (-0.9131658214789609+0j)
min energy found: (-1.018267970453952+0j)
updated theta: [0.0, 13.764, 0.0, 0.0, -0.5]
theta: [0.0, 13.764, 0.0, 0.0, -0.5]
working on theta[1]
job id: 63161ef27a1fe6f4770a1fda
energy: (-1.0080514078454523+0j)
energy: (-1.0118251343090445+0j)
energy: (-1.012691245417299+0j)
energy: (-1.0118852657853517+0j)
energy: (-1.0169465943286793+0j)
energy: (-1.0214977166086967+0j)
energy: (-1.0204675163910986+0j)
energy: (-1.0177262193608383+0j)
energy: (-1.0178623174237134+0j)
energy: (-1.0163947854624114+0j)
energy: (-1.0142978336247765+0j)
min energy found: (-1.0214977166086967+0j)
updated theta: [0.0, 13.

job id: 631633bc43edaaefaeed157c
energy: (-1.0567071038340434+0j)
energy: (-1.0712076522837959+0j)
energy: (-1.0748891484083278+0j)
energy: (-1.0795418530011969+0j)
energy: (-1.0804077595907162+0j)
energy: (-1.0848363880523495+0j)
energy: (-1.0849000442898886+0j)
energy: (-1.087260012202367+0j)
energy: (-1.0908322293327226+0j)
energy: (-1.0936584496283366+0j)
energy: (-1.0861606540916253+0j)
energy: (-1.0888100179674756+0j)
energy: (-1.0780334706641133+0j)
energy: (-1.081736641903857+0j)
energy: (-1.0807238214806938+0j)
energy: (-1.071784501062204+0j)
energy: (-1.06659983093524+0j)
min energy found: (-1.0936584496283366+0j)
updated theta: [3.108, 11.1, -0.666, 0.0, 1.5]
theta: [3.108, 11.1, -0.666, 0.0, 1.5]
working on theta[3]
job id: 6316367743edaa4656ed158a
energy: (-0.8133430747227999+0j)
energy: (-0.8717169726427672+0j)
energy: (-0.9250902520318849+0j)
energy: (-0.9734590564037979+0j)
energy: (-1.0135967426670123+0j)
energy: (-1.0602733298958407+0j)
energy: (-1.073580746376374+0j)

In [124]:
hill_step_run_results1

{0.3: [{'job_id': '63147a346acc638139372751',
   'counts': [{'00': 264, '01': 9160, '10': 454, '11': 122},
    {'00': 752, '01': 4613, '10': 1688, '11': 2947},
    {'00': 231, '01': 9267, '10': 371, '11': 131},
    {'00': 1608, '01': 3547, '10': 3153, '11': 1692},
    {'00': 150, '01': 9383, '10': 340, '11': 127},
    {'00': 1563, '01': 3361, '10': 3095, '11': 1981},
    {'00': 155, '01': 9541, '10': 191, '11': 113},
    {'00': 1299, '01': 4133, '10': 2151, '11': 2417},
    {'00': 2752, '01': 6996, '10': 187, '11': 65},
    {'00': 1012, '01': 4531, '10': 1424, '11': 3033},
    {'00': 5644, '01': 4121, '10': 199, '11': 36},
    {'00': 304, '01': 5130, '10': 608, '11': 3958},
    {'00': 3937, '01': 5816, '10': 174, '11': 73},
    {'00': 383, '01': 4939, '10': 950, '11': 3728},
    {'00': 1438, '01': 8358, '10': 126, '11': 78},
    {'00': 1597, '01': 3620, '10': 2409, '11': 2374},
    {'00': 285, '01': 9557, '10': 87, '11': 71},
    {'00': 2179, '01': 2947, '10': 2777, '11': 2097},
    {'

In [125]:
dists = [1, 3, 1.5]

thetas_hist2 = {}
values_hist2 = {}
std_devs_hist2 = {}
energies_hist2 = {}
hill_step_run_results2 = {}
    
all_paulis = []
all_coeffs = []
all_nuclear_repulsion_energies = []

backend = backend_lagos

for dist in dists:
    print("#########################################")
    print("Solving for distance " + str(dist) + ":")
    print("#########################################")
    paulis, coeffs = get_paulis_and_coeffs(dist)
    all_paulis.append(paulis)
    all_coeffs.append(coeffs)
    temp_res, min_energy, nuclear_repulsion_energy = get_exact_energies(dist)
    all_nuclear_repulsion_energies.append(nuclear_repulsion_energy)
    target_energy = min_energy

    initial_theta = [10 * dt, 62 * dt, 0, 0., 0.]
    
    parameter_search_space = {
        "0": np.linspace(20, -20, 21),
        "1": np.linspace(30, -30, 21),
        "2": np.linspace(30, -30, 21),
        "3": np.linspace(4, -4, 17),
        "4": np.linspace(4, -4, 17)
    }    
    
    parameters_order = [0, 1, 4, 3, 2]

    thetas = []
    values = []
    std_devs = []
    energies = []

    tic = time.perf_counter()
    
    
    cur_hill_step_run_results = []   

    cur_theta = copy.deepcopy(initial_theta)
    cur_energy = 10
    
    for i in range(4):
        if i >= 2:
            parameter_search_space = {
                "0": np.linspace(5, -5, 11),
                "1": np.linspace(5, -5, 11),
                "2": np.linspace(5, -5, 11),
                "3": np.linspace(0.5, -0.5, 11),
                "4": np.linspace(0.5, -0.5, 11)
            }
        for param_index in parameters_order:
            cur_hill_step_results = {}
            cur_energies = []
            cur_std_devs = []
            test_circs = []
            print("theta: " + str(cur_theta))
            print("working on theta[" + str(param_index) + "]")
            for param in parameter_search_space[str(param_index)]:
                test_theta = copy.deepcopy(cur_theta)
                if param_index in [0, 1, 2]:
                    param = param * dt
                test_theta[param_index] += param
                test_ansatz = ansatz_gate_inspired_weighted_real_device(test_theta, 0.222, backend, [5,6])
                circuits = create_circuits(paulis[3:], test_ansatz, [5,6])
                test_circs += circuits
            tran_circ_lagos = qiskit.transpile(test_circs, backend, initial_layout=[5,6])
            test_job_lagos = backend.run(tran_circ_lagos, shots=10000)
            cur_job_id = test_job_lagos.job_id()
            print("job id: " + str(cur_job_id))
            cur_hill_step_results["job_id"] = cur_job_id
            cur_results = test_job_lagos.result()
            cur_counts = cur_results.get_counts()
            cur_hill_step_results["counts"] = cur_counts
            for i in range(0, len(cur_counts), 2):
                temp_counts = [cur_counts[i], cur_counts[i], cur_counts[i], cur_counts[i+1]]
                expval, stddev = paulis_expectation_values(temp_counts, paulis[1:], coeffs[1:])
                print("energy: " + str(expval.real + coeffs[0] + nuclear_repulsion_energy))
                cur_energies.append(expval.real + coeffs[0] + nuclear_repulsion_energy)
                cur_std_devs.append(stddev)
            
            cur_hill_step_results["energies"] = cur_energies
            temp_min_energy = np.min(cur_energies)
            print("min energy found: " + str(temp_min_energy))
            cur_hill_step_results["min_energy"] = temp_min_energy
            opt_energy_index = cur_energies.index(temp_min_energy)
            cur_hill_step_results["std"] = cur_std_devs[opt_energy_index]
            if temp_min_energy < cur_energy:
                cur_energy = temp_min_energy
                if param_index in [0, 1, 2]:
                    multiplier = dt
                else:
                    multiplier = 1
                cur_theta[param_index] = cur_theta[param_index] + \
                    parameter_search_space[str(param_index)][opt_energy_index] * multiplier
            cur_hill_step_results["theta"] = copy.deepcopy(cur_theta)
            print("updated theta: " + str(cur_theta))
            # save results
            values.append(temp_min_energy - coeffs[0] - nuclear_repulsion_energy)
            energies.append(temp_min_energy)
            std_devs.append(cur_std_devs[opt_energy_index])
            thetas.append(copy.deepcopy(cur_theta))
            cur_hill_step_run_results.append(cur_hill_step_results)
    
    toc = time.perf_counter()

    thetas_hist2[dist] = copy.deepcopy(thetas)
    values_hist2[dist] = copy.deepcopy(values)
    std_devs_hist2[dist] = copy.deepcopy(std_devs)
    energies_hist2[dist] = copy.deepcopy(energies)
    hill_step_run_results2[dist] = copy.deepcopy(cur_hill_step_run_results)

    opt_theta_index = values.index(np.min(values))

    print(f"Finished the VQE in {toc - tic:0.4f} seconds")
    print("final theta: " + str(thetas[opt_theta_index]))
    res = min(energies)
    print("min eigen_value found: " + str(res))
    print("absolute error: " + str(np.abs(res - target_energy)))
    print("std: " + str(std_devs[opt_theta_index]))
    print("number of iterations: " + str(i))
    opt_sched = ansatz_gate_inspired_weighted_real_device(thetas[opt_theta_index], dt,
                                                      backend, qubits)
    print("solution schedule duration: " + str(opt_sched.duration * dt) + " [ns]")
    
all_results = {"thetas": thetas_hist2,
              "values": values_hist2,
              "std_devs": std_devs_hist2,
              "energies": energies_hist2,
              "run_results": hill_step_run_results2}

with open("./h2_results_hill_lagos2.pkl", 'ab') as pickle_results_file:
    pickle.dump(all_results, pickle_results_file)

#########################################
Solving for distance 1:
#########################################
theta: [2.22, 13.764, 0, 0.0, 0.0]
working on theta[0]
job id: 6316557d753db610fc124cb9
energy: (-1.0465542942863686+0j)
energy: (-1.0729224656915197+0j)
energy: (-1.069701605464692+0j)
energy: (-1.0704423619491443+0j)
energy: (-0.9277547171468495+0j)
energy: (-0.7866496343768014+0j)
energy: (-0.8734717977510732+0j)
energy: (-1.010831353740124+0j)
energy: (-1.0678701781391307+0j)
energy: (-1.0660712913617014+0j)
energy: (-1.033373213408236+0j)
energy: (-1.0181354816990669+0j)
energy: (-1.0330419819979255+0j)
energy: (-1.035511061329753+0j)
energy: (-1.0028314807748635+0j)
energy: (-1.0586862921255737+0j)
energy: (-0.9989422502826025+0j)
energy: (-0.9688971967285956+0j)
energy: (-0.9938013137854192+0j)
energy: (-1.0169042938507347+0j)
energy: (-1.0338066931938243+0j)
min energy found: (-1.0729224656915197+0j)
updated theta: [6.216, 13.764, 0, 0.0, 0.0]
theta: [6.216, 13.764, 0, 0.

job id: 63165da6b11a6382ce591e23
energy: (-0.9430135435112231+0j)
energy: (-0.9505490878731483+0j)
energy: (-0.976213799328944+0j)
energy: (-1.0541922811412254+0j)
energy: (-1.079868378783475+0j)
energy: (-1.0843185402059579+0j)
energy: (-1.0640168100584146+0j)
energy: (-1.0490667752126366+0j)
energy: (-1.0622747582294156+0j)
energy: (-1.0598530118983271+0j)
energy: (-0.9895433356261506+0j)
min energy found: (-1.0843185402059579+0j)
updated theta: [6.216, 13.764, 0, 0.5, -2.0]
theta: [6.216, 13.764, 0, 0.5, -2.0]
working on theta[1]
job id: 63165e28b11a633ec9591e26
energy: (-1.079050106737443+0j)
energy: (-1.0792187844668764+0j)
energy: (-1.0820481085064535+0j)
energy: (-1.078460988353851+0j)
energy: (-1.0831841275263163+0j)
energy: (-1.0856866975456931+0j)
energy: (-1.0775153171267746+0j)
energy: (-1.0867395912781377+0j)
energy: (-1.0849602101158848+0j)
energy: (-1.0857036791735446+0j)
energy: (-1.087757165947056+0j)
min energy found: (-1.087757165947056+0j)
updated theta: [6.216, 13.

job id: 631697f52a446d0c607ea75f
energy: (-0.7033669029725989+0j)
energy: (-0.7186417156774736+0j)
energy: (-0.7331150699749357+0j)
energy: (-0.7430560773213697+0j)
energy: (-0.756618611150647+0j)
energy: (-0.7574024132981164+0j)
energy: (-0.7661601675706097+0j)
energy: (-0.7656088000810541+0j)
energy: (-0.7669626219947293+0j)
energy: (-0.7652640405646968+0j)
energy: (-0.753890770863677+0j)
energy: (-0.7490441073679224+0j)
energy: (-0.7341958373549081+0j)
energy: (-0.7289597563441199+0j)
energy: (-0.7103429562596781+0j)
energy: (-0.6951295447068881+0j)
energy: (-0.6783693575010382+0j)
min energy found: (-0.7669626219947293+0j)
updated theta: [6.216, 13.764, 0, 0.0, -1.0]
theta: [6.216, 13.764, 0, 0.0, -1.0]
working on theta[2]
job id: 6316986343edaa6b00ed1751
energy: (-0.7661218526837699+0j)
energy: (-0.766078874906625+0j)
energy: (-0.7655111075516572+0j)
energy: (-0.7723208204280246+0j)
energy: (-0.7702718176527759+0j)
energy: (-0.766556787244977+0j)
energy: (-0.7646385042764943+0j)
e

job id: 6316a4c02a446d30c77ea7ab
energy: (-0.7576295673465836+0j)
energy: (-0.7314971112511989+0j)
energy: (-0.7438610196354326+0j)
energy: (-0.7663033275890738+0j)
energy: (-0.7822432743872729+0j)
energy: (-0.7901171867996905+0j)
energy: (-0.7697191102336738+0j)
energy: (-0.7671103877934762+0j)
energy: (-0.7553917407136503+0j)
energy: (-0.7533889652856537+0j)
energy: (-0.7683885737370406+0j)
min energy found: (-0.7901171867996905+0j)
updated theta: [10.212, 13.764, 4.662, -0.5, 0.0]
theta: [10.212, 13.764, 4.662, -0.5, 0.0]
working on theta[1]
job id: 6316ace06395626163b3199d
energy: (-0.7902840863961482+0j)
energy: (-0.7863801989209055+0j)
energy: (-0.7891011231441396+0j)
energy: (-0.7864183447263895+0j)
energy: (-0.7856563871056301+0j)
energy: (-0.7839044100192716+0j)
energy: (-0.7856754786712539+0j)
energy: (-0.7899066390772398+0j)
energy: (-0.7864100277579114+0j)
energy: (-0.794315557855324+0j)
energy: (-0.7918529115803441+0j)
min energy found: (-0.794315557855324+0j)
updated thet

job id: 6316c48d9c9b7098dfe12320
energy: (-0.8956532486590039+0j)
energy: (-0.9072699177549131+0j)
energy: (-0.926818435115428+0j)
energy: (-0.9331070536591017+0j)
energy: (-0.9437887215655976+0j)
energy: (-0.9533046412552606+0j)
energy: (-0.9663805989158702+0j)
energy: (-0.9631722440119526+0j)
energy: (-0.9664870901765519+0j)
energy: (-0.9706821066339093+0j)
energy: (-0.9649307072631046+0j)
energy: (-0.9642634981796292+0j)
energy: (-0.9605297841534497+0j)
energy: (-0.9486765747032795+0j)
energy: (-0.9417440852645891+0j)
energy: (-0.92852359258052+0j)
energy: (-0.9194850848895846+0j)
min energy found: (-0.9706821066339093+0j)
updated theta: [6.216, 12.876, 0, 0.0, -1.0]
theta: [6.216, 12.876, 0, 0.0, -1.0]
working on theta[3]
job id: 6316c4fa7a1fe64a950a2356
energy: (-0.8827132388351542+0j)
energy: (-0.9081669564521024+0j)
energy: (-0.9278692545257328+0j)
energy: (-0.9406165129998669+0j)
energy: (-0.9542741832335688+0j)
energy: (-0.9633546203798462+0j)
energy: (-0.9683090135197969+0j)


In [126]:
hill_step_run_results2

{1: [{'job_id': '6316557d753db610fc124cb9',
   'counts': [{'00': 391, '01': 9008, '10': 473, '11': 128},
    {'00': 635, '01': 4557, '10': 1914, '11': 2894},
    {'00': 269, '01': 9221, '10': 369, '11': 141},
    {'00': 1678, '01': 3180, '10': 3535, '11': 1607},
    {'00': 145, '01': 9370, '10': 363, '11': 122},
    {'00': 1537, '01': 3304, '10': 3125, '11': 2034},
    {'00': 195, '01': 9486, '10': 204, '11': 115},
    {'00': 1378, '01': 3961, '10': 2111, '11': 2550},
    {'00': 2958, '01': 6791, '10': 182, '11': 69},
    {'00': 922, '01': 4473, '10': 1483, '11': 3122},
    {'00': 5515, '01': 4244, '10': 200, '11': 41},
    {'00': 373, '01': 4965, '10': 777, '11': 3885},
    {'00': 4031, '01': 5719, '10': 177, '11': 73},
    {'00': 532, '01': 4813, '10': 1165, '11': 3490},
    {'00': 1483, '01': 8299, '10': 143, '11': 75},
    {'00': 1634, '01': 3412, '10': 2619, '11': 2335},
    {'00': 283, '01': 9553, '10': 84, '11': 80},
    {'00': 2134, '01': 2917, '10': 2833, '11': 2116},
    {'00

In [127]:
dists = [0.2]

thetas_hist3 = {}
values_hist3 = {}
std_devs_hist3 = {}
energies_hist3 = {}
hill_step_run_results3 = {}
    
all_paulis = []
all_coeffs = []
all_nuclear_repulsion_energies = []

backend = backend_lagos

for dist in dists:
    print("#########################################")
    print("Solving for distance " + str(dist) + ":")
    print("#########################################")
    paulis, coeffs = get_paulis_and_coeffs(dist)
    all_paulis.append(paulis)
    all_coeffs.append(coeffs)
    temp_res, min_energy, nuclear_repulsion_energy = get_exact_energies(dist)
    all_nuclear_repulsion_energies.append(nuclear_repulsion_energy)
    target_energy = min_energy

    initial_theta = [10 * dt, 62 * dt, 0, 0., 0.]
    
    parameter_search_space = {
        "0": np.linspace(20, -20, 21),
        "1": np.linspace(30, -30, 21),
        "2": np.linspace(30, -30, 21),
        "3": np.linspace(4, -4, 17),
        "4": np.linspace(4, -4, 17)
    }    
    
    parameters_order = [0, 1, 4, 3, 2]

    thetas = []
    values = []
    std_devs = []
    energies = []

    tic = time.perf_counter()
    
    
    cur_hill_step_run_results = []   

    cur_theta = copy.deepcopy(initial_theta)
    cur_energy = 10
    
    for i in range(4):
        if i >= 2:
            parameter_search_space = {
                "0": np.linspace(5, -5, 11),
                "1": np.linspace(5, -5, 11),
                "2": np.linspace(5, -5, 11),
                "3": np.linspace(0.5, -0.5, 11),
                "4": np.linspace(0.5, -0.5, 11)
            }
        for param_index in parameters_order:
            cur_hill_step_results = {}
            cur_energies = []
            cur_std_devs = []
            test_circs = []
            print("theta: " + str(cur_theta))
            print("working on theta[" + str(param_index) + "]")
            for param in parameter_search_space[str(param_index)]:
                test_theta = copy.deepcopy(cur_theta)
                if param_index in [0, 1, 2]:
                    param = param * dt
                test_theta[param_index] += param
                test_ansatz = ansatz_gate_inspired_weighted_real_device(test_theta, 0.222, backend, [5,6])
                circuits = create_circuits(paulis[3:], test_ansatz, [5,6])
                test_circs += circuits
            tran_circ_lagos = qiskit.transpile(test_circs, backend, initial_layout=[5,6])
            test_job_lagos = backend.run(tran_circ_lagos, shots=10000)
            cur_job_id = test_job_lagos.job_id()
            print("job id: " + str(cur_job_id))
            cur_hill_step_results["job_id"] = cur_job_id
            cur_results = test_job_lagos.result()
            cur_counts = cur_results.get_counts()
            cur_hill_step_results["counts"] = cur_counts
            for i in range(0, len(cur_counts), 2):
                temp_counts = [cur_counts[i], cur_counts[i], cur_counts[i], cur_counts[i+1]]
                expval, stddev = paulis_expectation_values(temp_counts, paulis[1:], coeffs[1:])
                print("energy: " + str(expval.real + coeffs[0] + nuclear_repulsion_energy))
                cur_energies.append(expval.real + coeffs[0] + nuclear_repulsion_energy)
                cur_std_devs.append(stddev)
            
            cur_hill_step_results["energies"] = cur_energies
            temp_min_energy = np.min(cur_energies)
            print("min energy found: " + str(temp_min_energy))
            cur_hill_step_results["min_energy"] = temp_min_energy
            opt_energy_index = cur_energies.index(temp_min_energy)
            cur_hill_step_results["std"] = cur_std_devs[opt_energy_index]
            if temp_min_energy < cur_energy:
                cur_energy = temp_min_energy
                if param_index in [0, 1, 2]:
                    multiplier = dt
                else:
                    multiplier = 1
                cur_theta[param_index] = cur_theta[param_index] + \
                    parameter_search_space[str(param_index)][opt_energy_index] * multiplier
            cur_hill_step_results["theta"] = copy.deepcopy(cur_theta)
            print("updated theta: " + str(cur_theta))
            # save results
            values.append(temp_min_energy - coeffs[0] - nuclear_repulsion_energy)
            energies.append(temp_min_energy)
            std_devs.append(cur_std_devs[opt_energy_index])
            thetas.append(copy.deepcopy(cur_theta))
            cur_hill_step_run_results.append(cur_hill_step_results)
    
    toc = time.perf_counter()

    thetas_hist18[dist] = copy.deepcopy(thetas)
    values_hist18[dist] = copy.deepcopy(values)
    std_devs_hist18[dist] = copy.deepcopy(std_devs)
    energies_hist18[dist] = copy.deepcopy(energies)
    hill_step_run_results18[dist] = copy.deepcopy(cur_hill_step_run_results)

    opt_theta_index = values.index(np.min(values))

    print(f"Finished the VQE in {toc - tic:0.4f} seconds")
    print("final theta: " + str(thetas[opt_theta_index]))
    res = min(energies)
    print("min eigen_value found: " + str(res))
    print("absolute error: " + str(np.abs(res - target_energy)))
    print("std: " + str(std_devs[opt_theta_index]))
    print("number of iterations: " + str(i))
    opt_sched = ansatz_gate_inspired_weighted_real_device(thetas[opt_theta_index], dt,
                                                      backend, qubits)
    print("solution schedule duration: " + str(opt_sched.duration * dt) + " [ns]")
    
all_results = {"thetas": thetas_hist3,
              "values": values_hist3,
              "std_devs": std_devs_hist3,
              "energies": energies_hist3,
              "run_results": hill_step_run_results3}

with open("./h2_results_hill_lagos3.pkl", 'ab') as pickle_results_file:
    pickle.dump(all_results, pickle_results_file)

#########################################
Solving for distance 0.2:
#########################################
theta: [2.22, 13.764, 0, 0.0, 0.0]
working on theta[0]
job id: 6316ea879c9b705c82e123cf
energy: (0.4312602835043995+0j)
energy: (0.31318236585523795+0j)
energy: (0.30211233323770603+0j)
energy: (0.27620491278709824+0j)
energy: (0.5789029609887661+0j)
energy: (1.2142838929775839+0j)
energy: (0.9805238714901672+0j)
energy: (0.5071010399455487+0j)
energy: (0.2527543184103571+0j)
energy: (0.23566135839070412+0j)
energy: (0.24190329350260864+0j)
energy: (0.3157405417296655+0j)
energy: (0.35253022802285194+0j)
energy: (0.29909298372332493+0j)
energy: (0.36996129822068546+0j)
energy: (0.2055284619023543+0j)
energy: (0.3552996422325645+0j)
energy: (0.48921635719266643+0j)
energy: (0.49583734663834944+0j)
energy: (0.2960962872663968+0j)
energy: (0.2608665996642934+0j)
min energy found: (0.2055284619023543+0j)
updated theta: [0.0, 13.764, 0, 0.0, 0.0]
theta: [0.0, 13.764, 0, 0.0, 0.0]
wo

job id: 6316f00c76f8e32efb95d88d
energy: (0.5201123716851295+0j)
energy: (0.29011997683928037+0j)
energy: (0.45232484022862174+0j)
energy: (0.3684649256867676+0j)
energy: (0.20124137923139607+0j)
energy: (0.20239390681879055+0j)
energy: (0.21756080411413992+0j)
energy: (0.5020444206560644+0j)
energy: (0.5977724815377448+0j)
energy: (0.46128991324514024+0j)
energy: (0.5370761839941229+0j)
min energy found: (0.20124137923139607+0j)
updated theta: [0.0, 13.764, 0, 0.0, 0.5]
theta: [0.0, 13.764, 0, 0.0, 0.5]
working on theta[1]
job id: 6316f0c02a446de28d7ea92e
energy: (0.21942106524520533+0j)
energy: (0.2090695423889679+0j)
energy: (0.2058124334611211+0j)
energy: (0.20738151895376866+0j)
energy: (0.2062113968215349+0j)
energy: (0.2077544591367979+0j)
energy: (0.20833456111501958+0j)
energy: (0.20844073110545303+0j)
energy: (0.20730407237495285+0j)
energy: (0.21230978018981883+0j)
energy: (0.20750801893895776+0j)
min energy found: (0.2058124334611211+0j)
updated theta: [0.0, 13.764, 0, 0.0,

In [128]:
hill_step_run_results3

{0.2: [{'job_id': '6316ea879c9b705c82e123cf',
   'counts': [{'00': 558, '01': 8823, '10': 497, '11': 122},
    {'00': 608, '01': 4914, '10': 1434, '11': 3044},
    {'00': 206, '01': 9309, '10': 394, '11': 91},
    {'00': 1355, '01': 3739, '10': 2910, '11': 1996},
    {'00': 236, '01': 9325, '10': 331, '11': 108},
    {'00': 1848, '01': 3087, '10': 3443, '11': 1622},
    {'00': 241, '01': 9403, '10': 216, '11': 140},
    {'00': 1537, '01': 3955, '10': 2255, '11': 2253},
    {'00': 1936, '01': 7777, '10': 195, '11': 92},
    {'00': 1183, '01': 4357, '10': 1584, '11': 2876},
    {'00': 5345, '01': 4395, '10': 215, '11': 45},
    {'00': 317, '01': 5204, '10': 375, '11': 4104},
    {'00': 4149, '01': 5641, '10': 174, '11': 36},
    {'00': 104, '01': 5180, '10': 321, '11': 4395},
    {'00': 1691, '01': 8110, '10': 148, '11': 51},
    {'00': 929, '01': 4203, '10': 1797, '11': 3071},
    {'00': 375, '01': 9455, '10': 107, '11': 63},
    {'00': 1982, '01': 3023, '10': 2976, '11': 2019},
    {'0

In [129]:
dists = [2.5]

thetas_hist4 = {}
values_hist4 = {}
std_devs_hist4 = {}
energies_hist4 = {}
hill_step_run_results4 = {}
    
all_paulis = []
all_coeffs = []
all_nuclear_repulsion_energies = []

backend = backend_lagos

for dist in dists:
    print("#########################################")
    print("Solving for distance " + str(dist) + ":")
    print("#########################################")
    paulis, coeffs = get_paulis_and_coeffs(dist)
    all_paulis.append(paulis)
    all_coeffs.append(coeffs)
    temp_res, min_energy, nuclear_repulsion_energy = get_exact_energies(dist)
    all_nuclear_repulsion_energies.append(nuclear_repulsion_energy)
    target_energy = min_energy

    initial_theta = [10 * dt, 62 * dt, 0, 0., 0.]
    
    parameter_search_space = {
        "0": np.linspace(20, -20, 21),
        "1": np.linspace(30, -30, 21),
        "2": np.linspace(30, -30, 21),
        "3": np.linspace(4, -4, 17),
        "4": np.linspace(4, -4, 17)
    }    
    
    parameters_order = [0, 1, 4, 3, 2]

    thetas = []
    values = []
    std_devs = []
    energies = []

    tic = time.perf_counter()
    
    
    cur_hill_step_run_results = []   

    cur_theta = copy.deepcopy(initial_theta)
    cur_energy = 10
    
    for i in range(5):
        if i >= 2:
            parameter_search_space = {
                "0": np.linspace(5, -5, 11),
                "1": np.linspace(5, -5, 11),
                "2": np.linspace(5, -5, 11),
                "3": np.linspace(0.5, -0.5, 11),
                "4": np.linspace(0.5, -0.5, 11)
            }
        for param_index in parameters_order:
            cur_hill_step_results = {}
            cur_energies = []
            cur_std_devs = []
            test_circs = []
            print("theta: " + str(cur_theta))
            print("working on theta[" + str(param_index) + "]")
            for param in parameter_search_space[str(param_index)]:
                test_theta = copy.deepcopy(cur_theta)
                if param_index in [0, 1, 2]:
                    param = param * dt
                test_theta[param_index] += param
                test_ansatz = ansatz_gate_inspired_weighted_real_device(test_theta, 0.222, backend, [5,6])
                circuits = create_circuits(paulis[3:], test_ansatz, [5,6])
                test_circs += circuits
            tran_circ_lagos = qiskit.transpile(test_circs, backend, initial_layout=[5,6])
            test_job_lagos = backend.run(tran_circ_lagos, shots=10000)
            cur_job_id = test_job_lagos.job_id()
            print("job id: " + str(cur_job_id))
            cur_hill_step_results["job_id"] = cur_job_id
            cur_results = test_job_lagos.result()
            cur_counts = cur_results.get_counts()
            cur_hill_step_results["counts"] = cur_counts
            for i in range(0, len(cur_counts), 2):
                temp_counts = [cur_counts[i], cur_counts[i], cur_counts[i], cur_counts[i+1]]
                expval, stddev = paulis_expectation_values(temp_counts, paulis[1:], coeffs[1:])
                print("energy: " + str(expval.real + coeffs[0] + nuclear_repulsion_energy))
                cur_energies.append(expval.real + coeffs[0] + nuclear_repulsion_energy)
                cur_std_devs.append(stddev)
            
            cur_hill_step_results["energies"] = cur_energies
            temp_min_energy = np.min(cur_energies)
            print("min energy found: " + str(temp_min_energy))
            cur_hill_step_results["min_energy"] = temp_min_energy
            opt_energy_index = cur_energies.index(temp_min_energy)
            cur_hill_step_results["std"] = cur_std_devs[opt_energy_index]
            if temp_min_energy < cur_energy:
                cur_energy = temp_min_energy
                if param_index in [0, 1, 2]:
                    multiplier = dt
                else:
                    multiplier = 1
                cur_theta[param_index] = cur_theta[param_index] + \
                    parameter_search_space[str(param_index)][opt_energy_index] * multiplier
            cur_hill_step_results["theta"] = copy.deepcopy(cur_theta)
            print("updated theta: " + str(cur_theta))
            # save results
            values.append(temp_min_energy - coeffs[0] - nuclear_repulsion_energy)
            energies.append(temp_min_energy)
            std_devs.append(cur_std_devs[opt_energy_index])
            thetas.append(copy.deepcopy(cur_theta))
            cur_hill_step_run_results.append(cur_hill_step_results)
    
    toc = time.perf_counter()

    thetas_hist4[dist] = copy.deepcopy(thetas)
    values_hist4[dist] = copy.deepcopy(values)
    std_devs_hist4[dist] = copy.deepcopy(std_devs)
    energies_hist4[dist] = copy.deepcopy(energies)
    hill_step_run_results4[dist] = copy.deepcopy(cur_hill_step_run_results)

    opt_theta_index = values.index(np.min(values))

    print(f"Finished the VQE in {toc - tic:0.4f} seconds")
    print("final theta: " + str(thetas[opt_theta_index]))
    res = min(energies)
    print("min eigen_value found: " + str(res))
    print("absolute error: " + str(np.abs(res - target_energy)))
    print("std: " + str(std_devs[opt_theta_index]))
    print("number of iterations: " + str(i))
    opt_sched = ansatz_gate_inspired_weighted_real_device(thetas[opt_theta_index], dt,
                                                      backend, qubits)
    print("solution schedule duration: " + str(opt_sched.duration * dt) + " [ns]")
    
all_results = {"thetas": thetas_hist4,
              "values": values_hist4,
              "std_devs": std_devs_hist4,
              "energies": energies_hist4,
              "run_results": hill_step_run_results4}

with open("./h2_results_hill_lagos4.pkl", 'ab') as pickle_results_file:
    pickle.dump(all_results, pickle_results_file)

#########################################
Solving for distance 2.5:
#########################################
theta: [2.22, 13.764, 0, 0.0, 0.0]
working on theta[0]
job id: 6316ff1376f8e334bf95d8f7
energy: (-0.7719847201130665+0j)
energy: (-0.7912161391817802+0j)
energy: (-0.7849932655925036+0j)
energy: (-0.7681200474540428+0j)
energy: (-0.7468554717778615+0j)
energy: (-0.7014205959428743+0j)
energy: (-0.7128144731668802+0j)
energy: (-0.7457349150947292+0j)
energy: (-0.7464611606715605+0j)
energy: (-0.7315312259768332+0j)
energy: (-0.7246035374656483+0j)
energy: (-0.7214165593717818+0j)
energy: (-0.7239142184178531+0j)
energy: (-0.7224211070184423+0j)
energy: (-0.707853182942976+0j)
energy: (-0.7027342739972497+0j)
energy: (-0.6926321166711646+0j)
energy: (-0.6905850197290799+0j)
energy: (-0.6849522772151486+0j)
energy: (-0.6793602478022105+0j)
energy: (-0.6770456377089493+0j)
min energy found: (-0.7912161391817802+0j)
updated theta: [6.216, 13.764, 0, 0.0, 0.0]
theta: [6.216, 13.764, 

job id: 631706466395628ecab31b6b
energy: (-0.6770306340074577+0j)
energy: (-0.6582654650983475+0j)
energy: (-0.627814879655838+0j)
energy: (-0.6284044048975774+0j)
energy: (-0.80711338254374+0j)
energy: (-0.8184951876644173+0j)
energy: (-0.8112989647787026+0j)
energy: (-0.8123961491311522+0j)
energy: (-0.7851757244226534+0j)
energy: (-0.7604530164248482+0j)
energy: (-0.7484768334048872+0j)
min energy found: (-0.8184951876644173+0j)
updated theta: [6.66, 16.428, 1.998, 0.0, -2.0]
theta: [6.66, 16.428, 1.998, 0.0, -2.0]
working on theta[1]
job id: 6317069676e4d143a60b454d
energy: (-0.8190283220914869+0j)
energy: (-0.8150663674281386+0j)
energy: (-0.8115690442129821+0j)
energy: (-0.813480480228757+0j)
energy: (-0.8162143909385589+0j)
energy: (-0.8181008806366679+0j)
energy: (-0.8191107298524996+0j)
energy: (-0.8186833353699967+0j)
energy: (-0.8205374826273886+0j)
energy: (-0.8159063202490122+0j)
energy: (-0.8168315944282833+0j)
min energy found: (-0.8205374826273886+0j)
updated theta: [6.

In [130]:
hill_step_run_results4

{2.5: [{'job_id': '6316ff1376f8e334bf95d8f7',
   'counts': [{'00': 571, '01': 8840, '10': 480, '11': 109},
    {'00': 545, '01': 4942, '10': 1438, '11': 3075},
    {'00': 186, '01': 9362, '10': 375, '11': 77},
    {'00': 1387, '01': 3829, '10': 2832, '11': 1952},
    {'00': 176, '01': 9426, '10': 327, '11': 71},
    {'00': 1743, '01': 3173, '10': 3367, '11': 1717},
    {'00': 187, '01': 9396, '10': 246, '11': 171},
    {'00': 1498, '01': 4029, '10': 2207, '11': 2266},
    {'00': 1883, '01': 7804, '10': 203, '11': 110},
    {'00': 1161, '01': 4309, '10': 1697, '11': 2833},
    {'00': 5279, '01': 4487, '10': 208, '11': 26},
    {'00': 285, '01': 5145, '10': 371, '11': 4199},
    {'00': 4121, '01': 5670, '10': 170, '11': 39},
    {'00': 78, '01': 5301, '10': 301, '11': 4320},
    {'00': 1649, '01': 8140, '10': 159, '11': 52},
    {'00': 913, '01': 4249, '10': 1701, '11': 3137},
    {'00': 464, '01': 9333, '10': 141, '11': 62},
    {'00': 2059, '01': 3014, '10': 2834, '11': 2093},
    {'00